In [1]:
import pandas as pd
import gzip
import pickle
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
clinical_data_path = '/Users/mohammad.enayet/Documents/Personal/Duke-Dataset/Clinical_and_Other_Features_final_final.xlsx'
features_path = '/Users/mohammad.enayet/Documents/Personal/Duke-Dataset/features_extracted_VGG16.pkl.gz'

In [3]:
clinical_data = pd.read_excel(clinical_data_path)

# Rename columns for consistency
clinical_data.columns = [col.replace(' ', '_') for col in clinical_data.columns]
clinical_data = clinical_data.rename(columns={'Patient_ID': 'patient_id', 'Pathologic_Response_to_Neoadjuvant_Therapy': 'response'})

# Convert 'Breast_MRI_001' to '001' in the patient_id column
clinical_data['patient_id'] = clinical_data['patient_id'].apply(lambda x: x.split('_')[-1] if isinstance(x, str) else x)

# Convert response to binary: 1 or 2 means response, anything other than 1 or 2 means no response
clinical_data['response'] = clinical_data['response'].apply(lambda x: 1 if x in [1, 2] else 0)

In [4]:
clinical_data

,patient_id,Date_of_Birth_(Days),Menopause_(at_diagnosis),Race_and_Ethnicity,ER,PR,HER2,Mol_Subtype,Staging(Nodes),Staging,Multicentric/Multifocal,Contralateral_Breast_Involvement,Lymphadenopathy_or_Suspicious_Nodes,Skin/Nipple_Invovlement,response
0,001,-15209,0,2,0,0,1,2,1,0,0,0,0,0,0
1,002,-14061,0,2,0,0,0,3,0,0,0,0,0,0,1
2,005,-13932,0,5,1,0,1,1,1,0,1,0,1,0,1
3,009,-20541,1,1,0,0,0,3,0,0,1,0,0,0,1
4,010,-24712,1,1,0,0,0,3,2,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,285,-17088,0,2,1,1,0,0,1,0,1,0,0,0,1
101,287,-16980,0,1,1,1,0,0,1,0,1,0,0,0,0
102,290,-14768,0,1,1,1,0,0,1,0,1,1,1,0,0
103,293,-18324,0,1,0,0,0,3,1,-1,1,0,0,1,0


In [3]:
# Load the features from the compressed file

with gzip.open(features_path, 'rb') as f:
    features_without_pyspark = pickle.load(f)

# Extract patient IDs from the keys
features_data = {
    'patient_id': [key.split('-')[0] for key in features_without_pyspark.keys()],
    **{f'feature_{i}': [features_without_pyspark[key][i] for key in features_without_pyspark.keys()] 
       for i in range(len(next(iter(features_without_pyspark.values()))))}
}

# Create a DataFrame for the features
features_df = pd.DataFrame(features_data)

In [4]:
features_df

,patient_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_25078,feature_25079,feature_25080,feature_25081,feature_25082,feature_25083,feature_25084,feature_25085,feature_25086,feature_25087
0,717,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8.039151,0.0
1,132,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,...,7.669905,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,10.982861,0.0
2,285,0.0,0.0,0.0,0.0,0.0,0.0,4.478312,0.0,0.0000,...,16.031435,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4.292235,0.0
3,258,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,3.6522,...,29.501215,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.992551,0.0
4,850,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,...,14.871337,2.640998,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8788,577,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,3.883359,0.0
8789,041,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,...,9.369884,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.450093,0.0
8790,717,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,10.127481,0.0
8791,198,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,...,6.256354,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.811845,0.0


In [6]:
def plot_auroc(y_true, y_scores, model_name):
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{model_name} Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'{model_name} Confusion Matrix')
    plt.show()

    tn, fp, fn, tp = cm.ravel()
    print(f"True Positives: {tp}")
    print(f"False Positives: {fp}")
    print(f"True Negatives: {tn}")
    print(f"False Negatives: {fn}")

def plot_loss_curves(train_losses, val_losses, test_losses, model_name):
    iterations = range(1, len(train_losses) + 1)
    plt.figure()
    plt.plot(iterations, train_losses, label='Training Loss')
    plt.plot(iterations, val_losses, label='Validation Loss')
    plt.plot(iterations, test_losses, label='Test Loss')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.title(f'{model_name} Loss Curves')
    plt.legend()
    plt.show()

def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='accuracy')
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    
    plt.legend(loc="best")
    plt.show()

def evaluate_model_with_cross_val(model, X, y, cv=5):
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    print(f'Cross-validation scores: {scores}')
    print(f'Mean cross-validation score: {np.mean(scores)}')

def plot_feature_importance(model, feature_names):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]

    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(len(feature_names)), importances[indices], color="r", align="center")
    plt.xticks(range(len(feature_names)), feature_names, rotation=90)
    plt.xlim([-1, len(feature_names)])
    plt.show()

def train_and_evaluate(model, param_grid,X_pca,y, X_train, y_train, X_val, y_val, X_test, y_test, model_name):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    
    start_time = time.time()
    grid_result = grid_search.fit(X_train, y_train)
    end_time = time.time()
    training_time = end_time - start_time
    
    best_model = grid_result.best_estimator_
    
    # Validation Performance
    y_val_pred = best_model.predict(X_val)
    y_val_proba = best_model.predict_proba(X_val)[:, 1]
    val_accuracy = accuracy_score(y_val, y_val_pred)
    
    # Test Performance
    y_test_pred = best_model.predict(X_test)
    y_test_proba = best_model.predict_proba(X_test)[:, 1]
    test_accuracy = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    auroc = roc_auc_score(y_test, y_test_proba)
    
    print(f"{model_name} validation accuracy: {val_accuracy:.4f}")
    print(f"{model_name} test accuracy: {test_accuracy:.4f}")
    print(f"{model_name} precision: {precision:.4f}")
    print(f"{model_name} recall: {recall:.4f}")
    print(f"{model_name} F1 score: {f1:.4f}")
    print(f"{model_name} AUROC: {auroc:.4f}")
    print(f"{model_name} training time: {training_time:.4f} seconds")
    print(f"Best parameters for {model_name}: {grid_result.best_params_}")
    
    #plot_auroc(y_test, y_test_proba, model_name)
    #plot_confusion_matrix(y_test, y_test_pred, model_name)
    
    # Plot learning curve
    #plot_learning_curve(best_model, f"Learning Curves ({model_name})", X_pca, y, cv=5)
    
    # Cross-validation evaluation
    evaluate_model_with_cross_val(best_model, X_pca, y, cv=5)
    
    # Plot feature importance for ensemble models
    #if hasattr(best_model, 'feature_importances_'):
    #    plot_feature_importance(best_model, [f'pca_{i}' for i in range(X_pca.shape[1])])
    
    # Plot loss curves if the model supports it
    #if hasattr(best_model, 'staged_predict_proba'):
    #    train_losses = []
    #    val_losses = []
    #    test_losses = []
    #    for train_pred_proba, val_pred_proba, test_pred_proba in zip(best_model.staged_predict_proba(X_train),
    #                                                                 best_model.staged_predict_proba(X_val),
    #                                                                 best_model.staged_predict_proba(X_test)):
    #        train_loss = np.mean((y_train - train_pred_proba[:, 1]) ** 2)
    #        val_loss = np.mean((y_val - val_pred_proba[:, 1]) ** 2)
    #        test_loss = np.mean((y_test - test_pred_proba[:, 1]) ** 2)
    #        train_losses.append(train_loss)
    #        val_losses.append(val_loss)
    #        test_losses.append(test_loss)
    #    plot_loss_curves(train_losses, val_losses, test_losses, model_name)
    
    return best_model

In [7]:
# Ensure 'patient_id' columns are of the same type
features_df['patient_id'] = features_df['patient_id'].astype(str)
clinical_data['patient_id'] = clinical_data['patient_id'].astype(str)

# Combine extracted features with clinical data
combined_df = features_df.merge(clinical_data, on='patient_id', how='left')

# Check the combined DataFrame
print(combined_df.head())

# Check the shape of the combined DataFrame
print(f"Combined DataFrame shape: {combined_df.shape}")

Combined DataFrame shape: (3536, 25103)


In [8]:
## Assuming combined_df is your DataFrame
## Separate features and target

X = combined_df.drop(columns=['patient_id', 'response'])
y = combined_df['response']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform PCA
pca = PCA(n_components=35)  # Adjust the number of components as needed
X_pca = pca.fit_transform(X_scaled)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_pca, y, test_size=0.5, random_state=22)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=22)

In [9]:
# Logistic Regression
lr = LogisticRegression(max_iter=500)
param_grid_lr = {
    'C': [0.01, 0.1, 1.0, 10.0],
    'penalty': ['l2'],
    'solver': ['lbfgs']
}
train_and_evaluate(lr, param_grid_lr,X_pca,y, X_train, y_train, X_val, y_val, X_test, y_test, "Logistic Regression")

# Random Forest
rf = RandomForestClassifier()
param_grid_rf = {
    'n_estimators': [50, 100, 150, 200, 300],
    'max_depth': [5, 10, 15, 20, 30]
}
train_and_evaluate(rf, param_grid_rf,X_pca,y, X_train, y_train, X_val, y_val, X_test, y_test, "Random Forest")

# Gradient Boosted Trees
gbt = GradientBoostingClassifier()
param_grid_gbt = {
    'n_estimators': [50, 100, 150, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.5],
    'max_depth': [3, 5, 7]
}
train_and_evaluate(gbt, param_grid_gbt,X_pca,y, X_train, y_train, X_val, y_val, X_test, y_test, "Gradient Boosted Trees")

# Support Vector Machine
svm = SVC(probability=True)
param_grid_svm = {
    'C': [0.01, 0.1, 1.0, 10.0],
    'kernel': ['linear', 'rbf']
}
#train_and_evaluate(svm, param_grid_svm, X_train, y_train, X_val, y_val, X_test, y_test, "Support Vector Machine")

# XGBoost
xgb_model = xgb.XGBClassifier()
param_grid_xgb = {
    'n_estimators': [50, 100, 150, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.5],
    'max_depth': [3, 5, 7]
}
train_and_evaluate(xgb_model, param_grid_xgb,X_pca,y, X_train, y_train, X_val, y_val, X_test, y_test, "XGBoost")

# LightGBM
lgb_model = lgb.LGBMClassifier()
param_grid_lgb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}
#train_and_evaluate(lgb_model, param_grid_lgb, X_train, y_train, X_val, y_val, X_test, y_test, "LightGBM")


Logistic Regression validation accuracy: 0.8066
Logistic Regression test accuracy: 0.7986
Logistic Regression precision: 0.7538
Logistic Regression recall: 0.6378
Logistic Regression F1 score: 0.6910
Logistic Regression AUROC: 0.8800
Logistic Regression training time: 4.1804 seconds
Best parameters for Logistic Regression: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
Cross-validation scores: [0.77683616 0.78925035 0.7864215  0.77934936 0.79490806]
Mean cross-validation score: 0.7853530873668481
Random Forest validation accuracy: 0.9966
Random Forest test accuracy: 0.9989
Random Forest precision: 1.0000
Random Forest recall: 0.9968
Random Forest F1 score: 0.9984
Random Forest AUROC: 1.0000
Random Forest training time: 12.8921 seconds
Best parameters for Random Forest: {'max_depth': 15, 'n_estimators': 200}
Cross-validation scores: [0.97740113 0.99858557 0.99717115 0.99292786 0.99434229]
Mean cross-validation score: 0.9920856008118971
Gradient Boosted Trees validation accuracy: 0.9943
